# 📊 Análisis exploratorio: 💼 Adult Census Income Dataset  

## 1️⃣ **Entender el contexto del proyecto**

El análisis exploratorio que se va a realizar está basado en el dataset **Adult / Census Income**, disponible públicamente en el repositorio UCI Machine Learning:  
🔗 https://archive-beta.ics.uci.edu/dataset/2/adult  

Este dataset recoge información demográfica y laboral de más de 48.000 personas censadas en Estados Unidos, con el objetivo de predecir si sus ingresos anuales superan los **50.000 dólares**. Es uno de los conjuntos de datos más utilizados para estudiar tareas de **clasificación binaria**, especialmente dentro del aprendizaje automático.

---

### 🎯 **Objetivo del análisis**

El propósito principal de este análisis es **explorar y comprender cómo distintos factores socioeconómicos y demográficos influyen en el nivel de ingresos de los individuos**.  

A través del análisis exploratorio (EDA) buscaremos:

- Detectar patrones relacionados con el nivel educativo, el tipo de ocupación, el estado civil y otras características relevantes.  
- Identificar posibles **valores atípicos**, inconsistencias o datos faltantes que puedan afectar a nuestros modelos.  
- Preparar adecuadamente el dataset para trabajar con algoritmos de machine learning orientados a **clasificación binaria**.

Posteriormente implementaremos y compararemos distintos modelos de clasificación, entre ellos:

- **Regresión logística**
- **Árboles de decisión y Random Forest**
- **KNN, Naive Bayes y SVM**
- **Redes neuronales básicas (MLP)**  

Con este proyecto buscamos construir modelos predictivos robustos que permitan **estimar si una persona ganará más o menos de 50.000$ al año**, a partir de sus características personales y laborales.

---

### 🧩 **Significado de las variables**

| **Columna** | **Descripción** |
|-------------|-----------------|
| `age` | Edad del individuo. |
| `workclass` | Tipo de empleo (autónomo, sector público, privado…). |
| `fnlwgt` | Peso muestral asignado por el censo. |
| `education` | Nivel educativo completado. |
| `education-num` | Nivel educativo en formato numérico. |
| `marital-status` | Estado civil. |
| `occupation` | Ocupación laboral. |
| `relationship` | Relación del individuo dentro del hogar. |
| `race` | Raza/etnia. |
| `sex` | Sexo. |
| `capital-gain` | Ganancias de capital (inversiones). |
| `capital-loss` | Pérdidas de capital. |
| `hours-per-week` | Horas trabajadas por semana. |
| `native-country` | País de origen. |
| `income` | Variable objetivo: >50K o <=50K. |

---

🗒️ *Fuente de los datos: UCI Machine Learning Repository, Adult Dataset (1996).*  

## 2️⃣ **Importación de librerías y carga del dataset**

In [1]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento y modelos (scikit-learn)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos que usaremos más adelante
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Métricas
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report
)

Cargamos el dataset y visualizamos las primeras filas

In [2]:
columnas = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

dataset = './data/adult.data'

df = pd.read_csv(dataset, names=columnas, sep=',')

df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## 3️⃣ **Exploración inicial del dataset (EDA básico)**
- .info(), .describe(), .nunique()
- Dimensiones, tipos de datos, distribución inicial.

In [ ]:
# Dimensiones del dataset
df.shape

(32561, 15)

In [ ]:
# Información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [ ]:
# Estadísticas generales
df.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
# Número de valores únicos por columna
df.nunique()

age                  73
workclass             9
fnlwgt            21648
education            16
education-num        16
marital-status        7
occupation           15
relationship          6
race                  5
sex                   2
capital-gain        119
capital-loss         92
hours-per-week       94
native-country       42
income                2
dtype: int64

In [8]:
# Revisamos la distribución inicial de la variable objetivo para saber si el dataset está desbalanceado
df['income'].value_counts()

income
<=50K    24720
>50K      7841
Name: count, dtype: int64

## 4️⃣ **Limpieza de datos**

- Detección de nulos y tratamiento.

In [9]:
# Revisamos los valores nulos que pueda tener el dataset
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

- Eliminación de espacios en blanco.

In [36]:
df = df.apply(lambda col: col.str.strip() if col.dtype == 'object' else col)

- Eliminación o corrección de valores “?” o similares.

In [37]:
# Contar valores desconocidos '?'
valores_interrogacion = (df == '?').sum()

valores_interrogacion[valores_interrogacion > 0]

# Mostramos la cantidad de valores '?' por columna
print("Valores '?' por columna:")
print(valores_interrogacion[valores_interrogacion > 0])


Valores '?' por columna:
Series([], dtype: int64)


In [25]:
total_filas = len(df)

porcentajes_interrogaciones = (valores_interrogacion / total_filas) * 100

print(porcentajes_interrogaciones[valores_interrogacion > 0])

workclass         5.638647
occupation        5.660146
native-country    1.790486
dtype: float64


In [31]:
# Sacamos las columnas que contienen valores '?'
columnas_con_interrogacion = valores_interrogacion[valores_interrogacion > 0].index

# Mostramos cuántos valores distintos tiene cada una de esas columnas
for col in columnas_con_interrogacion:
    print(f"\nColumna: {col}")
    print(f"- Valores distintos (incluyendo '?'): {df[col].nunique()}")


Columna: workclass
- Valores distintos (incluyendo '?'): 9

Columna: occupation
- Valores distintos (incluyendo '?'): 15

Columna: native-country
- Valores distintos (incluyendo '?'): 42


In [34]:
for col in columnas_con_interrogacion:
    df[col] = df[col].replace('?', 'Unknown')

# Comprobar que ya no quedan '?'
print("Valores '?' restantes:")
print((df == '?').sum())

Valores '?' restantes:
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


## 5️⃣ **Análisis exploratorio profundo (EDA detallado)**

### 📌 **5.1. Distribuciones univariantes**
- Histogramas para numéricas.
- Countplots para categóricas.

### 📌 **5.2. Análisis bivariante**
- Ingresos vs educación.
- Ingresos vs horas trabajadas.
- Ingresos vs ocupación, sexo, raza…
- Mapas de calor de correlación.

## 6️⃣ **Preprocesamiento para Machine Learning**
- OneHotEncoding para variables categóricas.
- Escalado de numéricas (si lo necesita algún modelo).
- Dividir entre train y test.

## 7️⃣ **Modelado**

### 📌 **7.1. Regresión logística**
- Entrenamiento
- Reporte: accuracy, matriz de confusión, classification report.
- Ajuste de umbrales (FN, FP).

### 📌 **7.2. Árboles de clasificación**
- Entrenamiento simple
- Visualización del árbol
- Métricas

### 📌 **7.3. Random Forest (con optimización)**
- Modelo base
- RandomizedSearchCV
- Comparación base vs optimizado

### 📌 **7.4. KNN, Naive Bayes y SVM**
- Entrenar y evaluar cada uno
- Comparar métricas

### 📌 **7.5. MLP(Red neuronal básica)**
- Preprocesamiento especial si hace falta
- Arquitectura simple
- Evaluación

## 8️⃣ **Comparación final de modelos**
- Tabla comparativa de accuracy, precisión, recall, F1.
- Discusión sobre cuál es mejor y por qué.

## 9️⃣ **Conclusiones del proyecto**
- Hallazgos clave del EDA.
- Variables más influyentes.
- Limitaciones del dataset.
- Interpretación del mejor modelo.